# Modules used
<br>These modules come in handy to access some applications or manipulate data.
<br>
<br>`$ python -m pip install --upgrade pywin32`
<br>https://learn.microsoft.com/pt-br/office/vba/api/outlook.application
<br>win32com interacts with the Outlook desktop application (similar to VBA syntax)
<br>
<br>`$ pip install pandas`
<br>https://pandas.pydata.org/docs/
<br>Brings the ability to work with dataframes
<br>
<br>`$ pip install pdfplumber`
<br>https://pypi.org/project/pdfplumber
<br>pdfplumber come in handy to read ".pdf" archives

In [ ]:
import win32com.client as client
import datetime as datetime
import pandas as pd
import pdfplumber
import re
import os

from pathlib import Path

# Global variables

In [ ]:
## Folder where the automation is located
dir = Path(__file__).parent.resolve()

## Month/Year reference as emails are sent each month and kept in the same folder
date = datetime.date.today().strftime("%m/%Y")

## Outlook connection + parameters
outlook = client.Dispatch("Outlook.Application")
namespace = outlook.GetNamesPace('MAPI')
account = 'company@email.com'
folder = 'folder'
sender = 'partner@email.com'

# Save extract

In [ ]:
def get_extract(email, folder, sender):

    ## Selects the right account and folder
    account = namespace.Folders[email]
    inbox = account.Folders[folder]
    
    ## Searches for the email with the invoice
    BHS_email = [message for message in inbox.Items if message.SenderEmailAddress == sender and pd.to_datetime(message.ReceivedTime, utc = True).strftime("%m/%Y") == date]
    
    for message in BHS_email:
        ## Recovers only the attachments (win32com)
        attachments = message.Attachments

        for pdf in attachments:
            ## Selects the correct attachment
            if pdf.FileName[-8:] == "_ext.pdf":
                ## Saves the attachment
                pdf.SaveAsFile(dir / str(pdf.FileName))
                return pdf.filename

partner_pdf = get_extract(account, folder, sender)
if partner_pdf == None:
    print("No partner invoice, process finished")
    exit()
print("Invoice obtained")

# Read extract

In [ ]:
def read_extract(extract):

    ## Opens and reads the invoice (pdfplumber)
    with pdfplumber.open(dir / str(extract)) as pdf:
        
        ## Creates a DataFrame to store data (current blank)
        licenses_data = pd.DataFrame(columns=["Email","License","Status","price"])
        i = 0
        
        for pages in pdf.pages:
            
            ## Extracts the content from the invoice
            content = pages.extract_text()
            
            ## Uses RegEx to create a filter variable to find the licenses lines
            filter = re.compile(r'^Power|Plano|Exchange|Microsoft|Flow')
            
            for line in content.split('\n'):
                
                ## Filters only the licenses lines
                if filter.match(line):
                    
                    ## Everything before a "|" refers to wich license is
                    License = str(line.split('|')[0])  
                    
                    ## We have either "Atribuída" or "Disponível" in my case so it searches for these words based on "í"
                    status = re.findall(r'\w+í\w+', line)
                    status = str(status[0]) if status else None
                    
                    ## Email is the only word with a "@" and as the invoice gives the e-mail in two lines we use the "@" to get the username and later we add the domain
                    email = re.findall(r'\S+\w+@', line)
                    email = str(email[0] + "email.com") if email else None
                    
                    ## The str that represents the value is the oly one with a comma
                    price = re.findall(r'\d+,\d+', line)
                    price = float(price[0].replace(",", ".")) if price else None
                    
                    ## Setting the data in the dataframe
                    licenses_data.loc[i] = [
                        email,
                        License,
                        status,
                        price
                    ]
                    i += 1
                    
        return licenses_data

## Uses the filename of the extract to identify and read it
licenses = read_extract(partner_pdf)
print("Licenses identified")

# Read employee data

In [ ]:
def get_employees():
    
    ## All of the employees data comes from a spreadsheet obtained from other software
    doc = 'Employees.xlsx'

    ## The software in question returns a xlsx with the first row merged and the Headers at the second one
    ## Here we only read the cells after the first row and set the second row as header
    data = pd.read_excel(dir / doc)[1:]
    header = pd.read_excel(dir / doc).iloc[0]
    data.columns = header.values
    
    ## Retrieve only useful information
    user_data = data.loc[:, ['Cód. Interno', 'e-Mail', 'Dt. Desligamento']].reset_index(drop=True)      ## retorna apenas estas duas colunas ## .reset_index atualiza o index da planilha
    
    return user_data

employees = get_employees()
print("Employees data obtained")

# Analysis

In [ ]:
def do_analysis(invoice, users):
    
    ## Returns only the dismissed employees
    dismissed_users = users[~users['Dt. Desligamento'].isna()]
    
    ## Merges invoice and employees data to find which licesens shouldnt be paid
    dismissed_licensed = dismissed_invoice[~dismissed_invoice['Email'].isna()]
    dismissed_invoice = pd.merge(dismissed_users, invoice, how='inner', left_on='e-Mail', right_on='Email')
    dismissed_licensed = dismissed_licensed[['Cód. Interno', 'Dt. Desligamento', 'Email', 'License', 'Status', 'price']]
    ## Sums the price of these licenses
    dismissed_licensed_sum = dismissed_licensed[['price']].sum()
    
    ## Returns only the available licenses
    licenses_available = invoice[invoice['Status'] == "Disponível"]
    ## Sums the price of these licenses
    licenses_available_sum = licenses_available[['price']].sum()
    
    return dismissed_licensed, dismissed_licensed_sum, licenses_available, licenses_available_sum

results = do_analysis(licenses, employees)
print("Analysis done")

# Send report

In [ ]:
def send_report(analysis, invoice):

    ## Converting the tables to HTML as they're embeded in mail body
    dismissed = analysis[0].to_html(border = 1, index=False, justify="left")
    available = analysis[2].to_html(border = 1, index=False, justify="left")
    
    ## Returns sum price of both analysis
    sum_dismissed = analysis[1]['price']
    sum_available = analysis[3]['price']
    
    ## Get the invoice pdf
    attachment = str(dir / invoice)

    ## Creates the report as a Email, by default its going to send as your inbox
    mail = outlook.CreateItem(0)
    mail.To = 'company@email.com'
    mail.Subject = f"Microsoft assigned licenses analysis {date}"
    mail.Attachments.Add(attachment)
    mail.BodyFormat = 1
    mail.HTMLBody = """<body>
        <div>
            <p>
                Hello,
                <br/><br/>
                Within the {ref} invoice, the following licenses should have been disabled":
                <br/><br/>
                {dismissed_html}
                <br/>
                Adding up: R$ {sum_dismissed:.2f}
            </p>
        </div>
        <br/>
        <div>
            <p>
                These licenses are unassigned:
    		    <br/><br/>
                {available_html}
    		    <br/>
                Adding up: R$ {sum_available:.2f}
            </p>
        </div>
    	<br/>
        <div>
            <p>
                Best regards,
                <br/>
                Caio Pavesi (Automation)
            </p>
        </div>
    </body>""".format(dismissed_html = dismissed, sum_dismissed = sum_dismissed, available_html = available, sum_available = sum_available, ref = date)
    mail.Send()
    
    return None

send_report(results, partner_pdf)
print("Report sent")

# Folder cleanup

In [ ]:
def organize_folder(invoice):

    ## Deletes invoice
    os.remove(dir / invoice)

    return None

organize_folder(partner_pdf)
print("5S feito")